Simple Grid search
- find optimize G
- constant n
- for D days
- with out bet ratio

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle
from datetime import datetime, timedelta

In [3]:
#import data & count numbers of day
start_date = '15 Aug 2023'
end_date = '16 Aug 2023' # will get data before 17 Aug 2023
# Convert string dates to datetime objects
start_date_dt = datetime.strptime(start_date, '%d %b %Y')
end_date_dt = datetime.strptime(end_date, '%d %b %Y')
# Calculate the number of days (D)
D = (end_date_dt - start_date_dt).days
tick_data = dfb.get_tick_data(start_date,end_date)['EURUSD.mid']



2023-09-18 00:50:08,485 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2023-09-18 00:50:08,485 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURUSD
2023-09-18 00:50:08,511 - findatapy.market.datavendorweb - INFO - Downloading... 2023-08-15 00:00:00 https://www.dukascopy.com/datafeed/EURUSD/2023/07/15/00h_ticks.bi5
2023-09-18 00:50:14,035 - findatapy.market.datavendorweb - INFO - Completed request from Dukascopy
2023-09-18 00:50:16,096 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.
Optimal G: 0.0001
Optimal net profit: 94.63978880000002


In [19]:
k =5
ps = k*len(tick_data) #for day d
ps

540055

In [20]:
def get_daily_data(tick_data, d):
    current_date = start_date_dt + timedelta(days=d)
    return tick_data[tick_data.index.date == current_date.date()]
def convert_to_binomial(tick_data,grid_size,ladderized_function):
    ladderized_data = ladderized_function(tick_data,grid_size)
    jumps = dfb.filter_jumps(ladderized_data)
    binomial_data = jumps.diff()/grid_size
    binomial_data = np.vectorize(lambda x: float('{:.5g}'.format(x)))(binomial_data)
    return binomial_data[1:]
def compute_pnl_t(G,n,daily_data,d,position):
    B=dfb.convert_to_binomial(daily_data, G, dfb.ladderize_open)
    pnl = np.zeros(len(B))
    position_series = [position]*len(B) #for day d
    #position_series = np.zeros(len(B)) ## but for day d > 0 this will not be 0
    for t in np.arange(0,len(B)):
        position_series[t] = position_series[t-1] - n * B[t]
        pnl[t] = pnl[t-1] + position_series[t-1] * G * B[t]
    #pnl_T= pnl[-1] + position*(daily_data[-1]-G) # cl price - G
    pnl_T= pnl[-1] 
    position = position_series[-1]
    return pnl,pnl_T,position,position_series 
def net_profit(G,n,tick_data):
    profit = np.zeros(D)
    for d in range(D):
        daily_data = get_daily_data(tick_data, d)
        #B=dfb.convert_to_binomial(daily_data, G, dfb.ladderize_open)
        if d == 0:
            position = 0
            #position_series = np.zeros(len(B)) #equal 0 for first day
            pnl, pnl_T, position,position_series = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T
        else:
            #position_series = position * len(B)
            pnl, pnl_T, position,position_series = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T + profit[d-1]
    net_profit = profit[-1] 
    return net_profit, profit


def pnl_constraint(params, n, d):
    G = params[0]
    position = 0  # for the first day; may have to modify this
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, _ = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if pnl at any time t violates the constraint
    if any(p < -150000 for p in pnl):
        return -1
    return 0
# daily profit > -500K
def profit_constraint(params, n, tick_data):
    G = params[0]
    _, profit = net_profit(G, n, tick_data)
    
    if any(pf < -500000 for pf in profit):
        return -1 #violated
    return 0
# Constraint to ensure -10M < Position at any time t < 10M
def position_constraint(params, n, d):
    G = params[0]
    position = 0
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, position_series = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if position at any time t violates the constraint
    if any(abs(pos) > 10000000 for pos in position_series):
        return -1 #violated
    return 0
    

confuse about constraints

In [23]:
# daily profit > -500K
def profit_constraint(params, n, tick_data):
    G = params[0]
    _, profit = net_profit(G, n, tick_data)
    
    if any(pf < -500000 for pf in profit):
        return -1 #violated
    return 0
# Constraint to ensure -10M < Position at any time t < 10M
def position_constraint(params, n, d):
    G = params[0]
    daily_data = get_daily_data(tick_data, d)
    if d==0:
        position = 0
    else:
        _,_, position,_ = compute_pnl_t(G, n, daily_data, d, position)
    pnl, _,_, position_series = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if position at any time t violates the constraint
    if any(abs(pos) > 10000000 for pos in position_series):
        return -1 #violated
    return 0

# Constraint to ensure pnl_t at all time t > -150K

def pnl_constraint(params, n, d):
    G = params[0]
    daily_data = get_daily_data(tick_data, d)
    if d==0:
        position = 0  # for the first day
    else:
        _,_, position,_ = compute_pnl_t(G, n, daily_data, d, position)

    pnl, _,position, _ = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if pnl at any time t violates the constraint
    if any(p < -150000 for p in pnl):
        return -1
    return 0
def check_constraints(G, n, d):
    position_okay = position_constraint([G], n, d) == 0
    pnl_okay = pnl_constraint([G], n, d) == 0
    profit_okay = profit_constraint([G], n, tick_data) == 0
    return position_okay and pnl_okay and profit_okay
# Grid search
n=1000
best_profit = float('-inf')
best_G = None

# Custom list of values for G
#G_values = [0.0001, 0.001, 0.01, 0.1, 1]
G_values = [0.0001, 0.001]
# Grid Search

for G in G_values:
    for d in range(D):
        if check_constraints(G, n, d):
            current_profit,_ = net_profit(G, n, tick_data)
            if current_profit > best_profit:
                best_profit = current_profit
                best_G = G

if best_G is not None:
    print(f"Optimal G: {best_G}")
    print(f"Optimal net profit: {best_profit}")
else:
    print("No feasible solution found.")

Optimal G: 0.0001
Optimal net profit: 94.63978880000002


Nelder-Mead

In [6]:
def position_constraint(params, *args):
    G, n, d = params[0], args[0], args[1]
    position = 0
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, position_series = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if position at any time t violates the constraint
    if any(abs(pos) > 10000000 for pos in position_series):
        return -1 #violated
    return 0

def pnl_constraint(params, *args):
    G, n, d = params[0], args[0], args[1]
    position = 0  # for the first day; may have to modify this
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, _ = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if pnl at any time t violates the constraint
    if any(p < -150000 for p in pnl):
        return -1
    return 0

def profit_constraint(params, *args):
    G, n, tick_data = params[0], args[0], args[1]
    G = params[0]
    _, profit = net_profit(G, n, tick_data)
    
    if any(pf < -500000 for pf in profit):
        return -1 #violated
    return 0

In [8]:
from scipy.optimize import minimize

def neg_net_profit(params):
    G = params[0]
    # Getting the negative of the profit since we're minimizing
    net_profit_value, _ = net_profit(G, n, tick_data)
    return -net_profit_value

def combined_constraints(params):
    G = params[0]
    constraint_values = []
    for d in range(D):
        constraint_values.append(position_constraint(params, n, d))
        constraint_values.append(pnl_constraint(params, n, d))
    constraint_values.append(profit_constraint(params, n, tick_data))
    return sum(constraint_values)

# The main optimization
initial_guess = [0.0001]  
# We will incorporate the constraints into the objective by penalization
def objective_with_penalty(params):
    penalty_weight = 1e2  # you may need to adjust this value
    return neg_net_profit(params) + penalty_weight * abs(combined_constraints(params))

result = minimize(objective_with_penalty, initial_guess, method='Nelder-Mead', options={'xatol': 1e-4, 'disp': True})

# Display results
if result.success:
    optimal_G = result.x[0]
    print(f"Optimal G: {optimal_G}")
    print(f"Optimal (negative) net profit: {result.fun}")
else:
    print("Optimization did not converge!")

Optimization terminated successfully.
         Current function value: -253.960564
         Iterations: 32
         Function evaluations: 70
Optimal G: 1.2514214515686167e-05
Optimal (negative) net profit: -253.96056383100907
